![T2](https://techcrunch.com/wp-content/uploads/2009/02/terminator.jpg)

In [ ]:
import string
import random
import time
import subprocess
import crypt
import itertools

# Password Generator

In [ ]:
char_set = string.ascii_letters + string.digits + '!@#$%^&*()'

In [ ]:
print(char_set)

In [ ]:
print(len(char_set))

In [ ]:
def gen_password(length, char_set=None):
    if char_set == None:
        char_set = string.ascii_letters + string.digits + '!@#$%^&*()'
    password = ''
    for i in range(length):
        x = random.choice(char_set)
        password = password + x
    return password

In [ ]:
my_pass = gen_password(8)
print(my_pass)

# Password/PIN Cracker

In [ ]:
password_to_crack = '5536'

In [ ]:
count = 1
start_time = time.time()
while True:
    attempt = gen_password(len(password_to_crack), string.digits)
    if attempt == password_to_crack:
        print(f'Cracked Password "{attempt}" on Try {count:,} and took {time.time()-start_time:.4f} seconds')
        break
#     else:
#         print(f'Guess = {attempt} attempt # {count}')
    count = count+1

# Calculating Permutations with Repetition


Formula: 
- **R** is repetition count, eg password length
- **N** number of characters 

    **N**^**R** = Possible_Outcomes

For example a 4 digit PIN:

- R = 4
- N = 10 (0,1,2,3,4,5,6,7,8,9)

    10^4 = 10,000 possible outcomes

In [ ]:
char_set_len = 72
length = 8
print(f'Possilbe Permutations {char_set_len**length:,}')

# Sequential Cracking Method

In [ ]:
def crack_pass(password_to_crack, start_length=1, up_to_length=13, char_set=None):
    if char_set == None:
        char_set = string.ascii_letters + string.digits + '!@#$%^&*()'
    print(f'''
    Attempting to crack password up to {up_to_length} characters long using these charecters:
    {char_set}
    ''')
    cracked = False
    count = 1
    start_time = time.time()
    for length in range(start_length, up_to_length):
        if cracked != True:
            print(f'\tTrying with Password Length of {length} @ {time.time()-start_time:.4f} seconds')
            for i in itertools.product(char_set, repeat=length):
                attempt = ''.join(i)
                if attempt == password_to_crack:
                    print(f'\tCracked Password "{attempt}" on Try {count:,} and took {time.time()-start_time:.4f} seconds')
                    cracked = True
                    break
            #     else:
            #         print(f'Guess = {attempt} attempt # {count}')
                count = count+1
    if cracked:
        return 'Crack Success'
    else:
        return 'Crack Failed'

In [ ]:
password_to_crack = 'C@t5E'
password_to_crack

In [ ]:
crack_pass(password_to_crack)

# Cracking a Linux Password

In [ ]:
output = subprocess.run('sudo cat /etc/shadow'.split(), capture_output=True)

In [ ]:
for line in output.stdout.decode().split('\n'):
    if 'testuser' in line:
        break
salt = line.split(':')[1].split('$')[2]
hashed = line.split(':')[1].split('$')[3]
complete = f'$6${salt}${hashed}'
print(complete)

In [ ]:
def gen_hash_sha512(password, salt):
    sha512_hash = crypt.crypt(password, f'$6${salt}')
    return sha512_hash

In [ ]:
def crack_hashed_pass(password_hash, salt, up_to_length=13, char_set=None):
    if char_set == None:
        char_set = string.ascii_letters + string.digits + '!@#$%^&*()'
    print(f'''
    Attempting to crack password up to {up_to_length} characters long using these charecters:
    {char_set}
    ''')
    cracked = False
    count = 1
    start_time = time.time()
    for length in range(up_to_length):
        if cracked != True:
            print(f'\tTrying with Password Length of {length} @ {time.time()-start_time:.4f} seconds')
            for i in itertools.product(char_set, repeat=length):
                attempt = ''.join(i)
                # Pass Attempt to gen_hash_sha512 function
                hashed_attempt = gen_hash_sha512(attempt, salt)
                if hashed_attempt == password_hash:
                    print(f'\tCracked Password "{attempt}" on Try {count:,} and took {time.time()-start_time:.4f} seconds')
                    cracked = True
                    break
            #     else:
            #         print(f'Guess = {attempt} attempt # {count}')
                count = count+1
    if cracked:
        return ('Crack Success', attempt)
    else:
        return 'Crack Failed'

In [ ]:
def crack_linux_password(username):
    # Find Encrypted Password Line in /etc/shadow
    output = subprocess.run('sudo cat /etc/shadow'.split(), capture_output=True)
    for line in output.stdout.decode().split('\n'):
        if username in line:
            print(line)
            break
    # Parse Line for Salt and Hashed Password        
    salt = line.split(':')[1].split('$')[2]
    hashed = line.split(':')[1].split('$')[3]
    complete = f'$6${salt}${hashed}'
    print(f'\nS = {salt}\nH = {hashed}')
    # Pass Salt and Hashed Password into crack_hashed_pass function
    result = crack_hashed_pass(complete, salt, up_to_length=13, char_set=None)
    return result

In [ ]:
crack_linux_password('testuser')